# Jenn Wong
jennwong@berkeley.edu

Data was gathered from Everlane Mode SQL database from running

select * from {database};

In [2]:
import pandas as pd

import warnings
warnings.filterwarnings('ignore')

In [10]:
users_df = pd.read_csv('users.csv')
orders_df = pd.read_csv('orders.csv')
products_df = pd.read_csv('products.csv')

In [11]:
users_df.head()

,user_id,discovery_channel_category,discovery_channel,discovery_time,discovery_platform,location,gender
0,1957598,paid,other,2017-05-10 07:23:23,desktop,US,female
1,1771115,paid,affiliate,2017-02-15 16:33:02,mobile,US,female
2,1489936,direct,NaN,2016-09-13 16:07:53,desktop,US,female
3,2356025,paid,search,2017-10-12 17:13:53,desktop,US,female
4,1463297,organic search,NaN,2016-08-17 07:06:25,desktop,international,female


In [39]:
orders_df.head()

# this is a string and should be date-time for easier analysis
orders_df.completed_at = pd.to_datetime(orders_df.completed_at) 

In [13]:
products_df.head()

,style_id,style,category,launch_date
0,592,The Premium Wool Flannel Pinstripe Track Pant,Bottoms,2016-09-19 00:00:00
1,572,The Luxe Drape Cropped Scoop-Neck Tee,Knit Tops,2016-07-26 00:00:00
2,645,The Cashmere Cardigan,Sweaters,2016-10-10 00:00:00
3,435,The Modern Flannel Shirt,Shirting,2016-02-25 00:00:00
4,874,The Pleated Midi Skirt,Bottoms,2017-08-24 00:00:00


In [16]:
products_df.info() # this shows me that I need to change the launch_date column into datetime


products_df.launch_date = pd.to_datetime(products_df.launch_date)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 455 entries, 0 to 454
Data columns (total 4 columns):
style_id       455 non-null int64
style          455 non-null object
category       455 non-null object
launch_date    455 non-null datetime64[ns]
dtypes: datetime64[ns](1), int64(1), object(2)
memory usage: 14.3+ KB


## Question 1: New Product Launches

1. How would you measure the strength of a new product?



I will be defining the 'strength' of the new product as number of sales in the first two weeks of launch (because I know that Everlane releases new products every week and constitutes a large part of its revenue). This will allow me to see how it performs as a 'new product' as well as an 'old-er product' and identify seasonal trends (if any) in the week.

In order to answer this question, I need to get metadata on the products that were ordered. I will be joining the orders_df with the products_df on 'style_id' in order to get the data that I need.

In [54]:
detailed_orders = pd.merge(orders_df, products_df, how = 'left', on  = 'style_id')

In [70]:
time_period = 14 # this is the number of days after launch that I care about

detailed_orders['time_since_launch'] = (detailed_orders.completed_at - detailed_orders.launch_date)


# i only care about the days since launch and not such a high resolution
def get_days(date_time_object):
    """
    this function takes a datetime object and returns the days
    """
    return date_time_object.days

detailed_orders['days_since_launch'] = detailed_orders.time_since_launch.apply(get_days)

new_orders = detailed_orders[detailed_orders.days_since_launch <= time_period]

print(f'I started with {len(detailed_orders)} number of orders. I now have {len(new_orders)} rows of recent orders')
# len(new_orders

I started with 43548 number of orders. I now have 6014 rows of recent orders


In [21]:
# so now I need to find the count of sales for each day over two weeks after launch for each product
new_orders

433


In [42]:
(detailed_orders.completed_at[0] - detailed_orders.launch_date[0]).days

338

#### Additionally, I know that new users drive revenue for Everlane as well. 
Therefore I think it's important to see how new_products fair for this segment of the market, and possibly compare it to not-new users as well. If there is a strong sales trend in non-new users, that could be an indicator of the 'strength' of the new product!